This notebook is also to show my thinking, therefore not all the cells would be necessary in an optimized code

## High-level overview

If not recognize early on, Lyme disease can have debilitating effects on the health of people. Therefore it is important to know how high the danger is for people contract Lyme disease. And as most people are very visual, it is important to take the raw data provided on the CDC page and turn it into something more visual. This is important for individual persons as well as for policy makers.

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import json

In [2]:
# importing libraries for visualization
#import plotly as py
#import plotly.figure_factory as ff
import plotly.express as px
import nbformat
from dash import Dash, dcc, html, Input, Output

## Description of Input Data and Exploratory Analysis

The dataset used is [Lyme disease public use aggregated data with geography](https://healthdata.gov/CDC/Lyme-disease-public-use-aggregated-data-with-geogr/n9dr-4pe8/about_data) describe in detail on the website. I used the [.csv file](https://data.cdc.gov/api/views/qtbi-xd4i/rows.csv?accessType=DOWNLOAD). The data is reported on a voluntary basis which means that the number of case can be unterreported. More details on the data will be discovered during the EDA.

In [3]:
# Importing data
# Source: https://healthdata.gov/CDC/Lyme-disease-public-use-aggregated-data-with-geogr/n9dr-4pe8/about_data
df = pd.read_csv('Lyme_disease_public_use_aggregated_data_with_geography__2008-2021.csv')
df.head()

,Year,State,FIPS,Case_status,Sex,Age_cat_yrs,Frequency
0,2008,AK,Suppressed,Confirmed,Female,20+,5
1,2008,AL,Suppressed,Confirmed,Suppressed,Suppressed,6
2,2008,AL,Suppressed,Probable,Suppressed,Suppressed,3
3,2008,AZ,04013,Confirmed,Male,0-19,2
4,2008,AZ,04013,Probable,Male,0-19,3


Variables in the data set are the Year, the State, FIPS, Case Status, Sex, Age category and Frequency, meaning number of cases per year.

FIPS stands in general for Federal Information Processing Standards and here for a unique five digit code given to each county in the US. The first two numbers identify the state, the last three numbers the county within this state. An overview of the United States FIPS Codes can be found [here](https://unicede.air-worldwide.com/unicede/unicede_us_fips_codes.html)

In [4]:
df.info() # info gives more info than describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40468 entries, 0 to 40467
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         40468 non-null  int64 
 1   State        40468 non-null  object
 2   FIPS         40468 non-null  object
 3   Case_status  40468 non-null  object
 4   Sex          40468 non-null  object
 5   Age_cat_yrs  40468 non-null  object
 6   Frequency    40468 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


Good, no really NaNs, every columns contains the same number of entries. And each column got assigned the right data type. This is especially import for the FIPS column, while it looks like a number, it is actually more of a symbol, as a leading zero contains information.

In [5]:
print(df.columns)

Index(['Year', 'State', 'FIPS', 'Case_status', 'Sex', 'Age_cat_yrs',
       'Frequency'],
      dtype='object')


To make the exploration easier I am defining a function which takes a dataframe and for each column counts the number of different value in this column.

In [6]:
# Defining function
def value_counts_df(df):
    """ Takes a dataframe and prints the value counts in each of the columns
    Parameters
    ----------
    df : dataframe

    Returns
    -------
    """
    for i in df.columns:
        print(df.value_counts([i]))
    return

Checking which values are in each of the columns.

In [7]:
value_counts_df(df)

Year
2017    3607
2019    3321
2016    3239
2015    3133
2021    3054
2018    3026
2013    2976
2014    2871
2011    2697
2012    2681
2009    2565
2010    2559
2020    2515
2008    2224
Name: count, dtype: int64
State     
PA            5566
WI            5256
NY            4834
MN            3378
NJ            2907
VA            2597
MD            2216
ME            1357
WV            1195
MA            1192
CT            1063
NH            1025
VT             930
MI             764
IA             661
OH             604
RI             552
NC             552
IL             548
FL             537
IN             397
DE             341
CA             334
TX             235
OR             178
SC             140
ND             136
DC             122
TN              92
WA              88
KY              87
Suppressed      83
AL              65
NV              59
UT              58
AZ              58
KS              56
GA              36
ID              31
NE              26
MT              

The year column looks fine, no other values except numbers and all the values are in the range between 2008 and 2021, not big data entry errors there.

There are 83 state with Suppressed, at the momentan I am deciding to drop these lines.


In [8]:
# 83 states with Suppressed, I can savely drop these line
df.drop(df[df['State'] == 'Suppressed'].index, inplace=True)
df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40385 entries, 0 to 40460
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         40385 non-null  int64 
 1   State        40385 non-null  object
 2   FIPS         40385 non-null  object
 3   Case_status  40385 non-null  object
 4   Sex          40385 non-null  object
 5   Age_cat_yrs  40385 non-null  object
 6   Frequency    40385 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.5+ MB


The FIPS columns is more important, as I want to show the data by county. Without a FIPS I have no way of nowing anything about the county.

In [9]:
# Taking a closer looks at the FIPS columns, what is behind 'Suppressed' and 'Unkown'
df_suppressed = df[df['FIPS'] == 'Suppressed']
df_suppressed

,Year,State,FIPS,Case_status,Sex,Age_cat_yrs,Frequency
0,2008,AK,Suppressed,Confirmed,Female,20+,5
1,2008,AL,Suppressed,Confirmed,Suppressed,Suppressed,6
2,2008,AL,Suppressed,Probable,Suppressed,Suppressed,3
15,2008,CA,Suppressed,Confirmed,Male,0-19,8
16,2008,CA,Suppressed,Confirmed,Female,20+,11
...,...,...,...,...,...,...,...
40164,2021,WI,Suppressed,Probable,Suppressed,0-19,1
40165,2021,WI,Suppressed,Probable,Suppressed,Suppressed,16
40458,2021,WV,Suppressed,Confirmed,Male,0-19,1
40459,2021,WV,Suppressed,Confirmed,Suppressed,Suppressed,13


In [10]:
# A lot of columns contain more than one Suppressed
value_counts_df(df_suppressed)

Year
2019    175
2021    169
2017    166
2016    165
2018    162
2011    157
2013    155
2020    151
2014    147
2012    146
2015    141
2010    140
2008    138
2009    138
Name: count, dtype: int64
State
FL       91
MI       83
NC       82
SC       81
WI       81
VA       79
IA       78
IN       76
NY       75
MN       74
CA       74
OR       74
OH       68
TX       68
TN       67
WV       65
WA       62
ND       61
MD       59
IL       58
VT       57
PA       50
AL       49
KS       47
ME       46
KY       45
UT       35
NH       35
AZ       33
GA       31
NV       30
ID       28
NJ       27
NE       24
MA       23
AK       21
MT       21
RI       19
SD       17
MO       16
NM        9
CT        8
AR        8
LA        7
DC        3
MS        2
WY        2
CO        1
Name: count, dtype: int64
FIPS      
Suppressed    2150
Name: count, dtype: int64
Case_status
Confirmed      1198
Probable        952
Name: count, dtype: int64
Sex       
Suppressed    1035
Female         571
Male      

There is a quite a large number of Suppressed in FIPS, how much is it of the full data set?

In [11]:
# Good question, what to do with them
print(f"Shape of df: {df.shape}")
print(f"Shape of df_suppressed: {df_suppressed.shape}")
print(f"Percentage of Suppressed: {((df_suppressed.shape[0]/df.shape[0])*100):.2f}%")

Shape of df: (40385, 7)
Shape of df_suppressed: (2150, 7)
Percentage of Suppressed: 5.32%


For the sake of simplicity, I will drop alle the lines with Suppressed in the FIPS. We will definitely loose information here, as we have states with over 100 cases, but for this project I will go ahead and use only lines where the data quality is good. From the suppressed df one could get some additional information because the State, Case Status and Frequency is always provided. One could do an analysis only based on states. But in this case one would check if some information might be double which is for me beyond the scope of this project.

In [12]:
df.drop(df[df['FIPS'] == 'Suppressed'].index, inplace=True)
df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38235 entries, 3 to 40457
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         38235 non-null  int64 
 1   State        38235 non-null  object
 2   FIPS         38235 non-null  object
 3   Case_status  38235 non-null  object
 4   Sex          38235 non-null  object
 5   Age_cat_yrs  38235 non-null  object
 6   Frequency    38235 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.3+ MB


In [13]:
value_counts_df(df)

Year
2017    3435
2019    3140
2016    3068
2015    2986
2021    2878
2018    2858
2013    2815
2014    2718
2011    2534
2012    2530
2009    2421
2010    2414
2020    2357
2008    2081
Name: count, dtype: int64
State
PA       5516
WI       5175
NY       4759
MN       3304
NJ       2880
VA       2518
MD       2157
ME       1311
MA       1169
WV       1130
CT       1055
NH        990
VT        873
MI        681
IA        583
OH        536
RI        533
IL        490
NC        470
FL        446
DE        341
IN        321
CA        260
TX        167
DC        119
OR        104
ND         75
SC         59
KY         42
NV         29
WA         26
TN         25
AZ         25
UT         23
AL         16
KS          9
GA          5
MT          4
ID          3
NM          2
NE          2
AK          2
Name: count, dtype: int64
FIPS   
Unknown    622
34027      168
34005      156
34031      155
34003      154
          ... 
54035        1
37085        1
19117        1
37013        1
01049    

The FIPS column still has Unknown, what to do with that?

In [14]:
# Taking a closer looks at the FIPS columns, what is behind 'Suppressed' and 'Unkown', now looking at 'Unkown'
df_unknown = df[df['FIPS'] == 'Unknown']
df_unknown

,Year,State,FIPS,Case_status,Sex,Age_cat_yrs,Frequency
90,2008,CT,Unknown,Confirmed,Male,0-19,135
91,2008,CT,Unknown,Confirmed,Male,20+,411
92,2008,CT,Unknown,Confirmed,Female,0-19,85
93,2008,CT,Unknown,Confirmed,Female,20+,382
94,2008,CT,Unknown,Confirmed,Suppressed,0-19,2
...,...,...,...,...,...,...,...
40153,2021,WI,Unknown,Confirmed,Male,20+,3
40154,2021,WI,Unknown,Confirmed,Male,Suppressed,1
40155,2021,WI,Unknown,Confirmed,Suppressed,Suppressed,2
40156,2021,WI,Unknown,Probable,Male,20+,3


In [15]:
value_counts_df(df_unknown)

Year
2016    65
2015    54
2019    54
2008    52
2013    52
2009    45
2017    43
2014    40
2011    39
2018    38
2021    38
2010    36
2020    36
2012    30
Name: count, dtype: int64
State
CT       100
VA        96
MA        77
IN        56
NH        47
WV        46
DE        43
KY        35
DC        29
WI        22
RI        15
TN        13
MN         8
NJ         8
SC         7
TX         7
VT         7
MT         4
MD         2
Name: count, dtype: int64
FIPS   
Unknown    622
Name: count, dtype: int64
Case_status
Confirmed      331
Probable       291
Name: count, dtype: int64
Sex       
Male          253
Female        235
Suppressed     80
Unknown        54
Name: count, dtype: int64
Age_cat_yrs
20+            302
0-19           169
Suppressed     117
Unknown         34
Name: count, dtype: int64
Frequency
1            111
3             67
2             63
5             44
4             31
            ... 
103            1
104            1
108            1
109            1
467     

Also dropping the lines with unknown FIPS from the data set.

In [16]:
# I will do the same thing for the unknown as for the suppressed, but it might be worthwhile also later looking at the unknown
df.drop(df[df['FIPS'] == 'Unknown'].index, inplace=True)
df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37613 entries, 3 to 40457
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         37613 non-null  int64 
 1   State        37613 non-null  object
 2   FIPS         37613 non-null  object
 3   Case_status  37613 non-null  object
 4   Sex          37613 non-null  object
 5   Age_cat_yrs  37613 non-null  object
 6   Frequency    37613 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.3+ MB


In [17]:
value_counts_df(df)

Year
2017    3392
2019    3086
2016    3003
2015    2932
2021    2840
2018    2820
2013    2763
2014    2678
2012    2500
2011    2495
2010    2378
2009    2376
2020    2321
2008    2029
Name: count, dtype: int64
State
PA       5516
WI       5153
NY       4759
MN       3296
NJ       2872
VA       2422
MD       2155
ME       1311
MA       1092
WV       1084
CT        955
NH        943
VT        866
MI        681
IA        583
OH        536
RI        518
IL        490
NC        470
FL        446
DE        298
IN        265
CA        260
TX        160
OR        104
DC         90
ND         75
SC         52
NV         29
WA         26
AZ         25
UT         23
AL         16
TN         12
KS          9
KY          7
GA          5
ID          3
NM          2
NE          2
AK          2
Name: count, dtype: int64
FIPS 
34027    168
34005    156
34031    155
34013    154
34003    154
        ... 
48367      1
18047      1
18167      1
51678      1
01049      1
Name: count, Length: 782, dtype:

Different idea for dealing with the Suppressed in FIPS, not used anymore but I am leaving it in because it shows some problems with automatic typecasting when reading a table and how to solve it.

In [18]:
# For converting the Suppressed in the FIPS column, downloading a table with the information about FIPS and states
# Source: https://healthdata.gov/CDC/Lyme-disease-public-use-aggregated-data-with-geogr/n9dr-4pe8/about_data , story behind it: https://towardsdatascience.com/the-ultimate-state-county-fips-tool-1e4c54dc9dff/
fips = pd.read_csv('fips2county.tsv', sep='\t')
fips.head()

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,CountyCBSA
0,1,1,Autauga,Alabama,1001,AL,AL | AUTAUGA,33860.0
1,1,3,Baldwin,Alabama,1003,AL,AL | BALDWIN,19300.0
2,1,5,Barbour,Alabama,1005,AL,AL | BARBOUR,21640.0
3,1,7,Bibb,Alabama,1007,AL,AL | BIBB,13820.0
4,1,9,Blount,Alabama,1009,AL,AL | BLOUNT,13820.0


Uupps, FIPS like 01 and 001 automatically got cast as integers and lost part of information. But they are really strings just looking like number. Checking the datatypes with info(), dtypes() would also work, but info() gives more information

In [19]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   StateFIPS     3143 non-null   int64  
 1   CountyFIPS_3  3143 non-null   int64  
 2   CountyName    3143 non-null   object 
 3   StateName     3143 non-null   object 
 4   CountyFIPS    3143 non-null   int64  
 5   StateAbbr     3143 non-null   object 
 6   STATE_COUNTY  3143 non-null   object 
 7   CountyCBSA    1835 non-null   float64
dtypes: float64(1), int64(3), object(4)
memory usage: 196.6+ KB


In [20]:
# Cleaning data
# FIPS column: adding FIPS column just for state, replacing Suppressed by FIPS state + 000
# Dictionary to avoid casting Fips into int
datatypes = {'StateFIPS': str, 'CountyFIPS_3': str, 'CountyFIPS': str}
fips = pd.read_csv('fips2county.tsv', sep='\t', dtype=datatypes)
fips.head()

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,CountyCBSA
0,01,001,Autauga,Alabama,01001,AL,AL | AUTAUGA,33860.0
1,01,003,Baldwin,Alabama,01003,AL,AL | BALDWIN,19300.0
2,01,005,Barbour,Alabama,01005,AL,AL | BARBOUR,21640.0
3,01,007,Bibb,Alabama,01007,AL,AL | BIBB,13820.0
4,01,009,Blount,Alabama,01009,AL,AL | BLOUNT,13820.0


In [21]:
fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   StateFIPS     3143 non-null   object 
 1   CountyFIPS_3  3143 non-null   object 
 2   CountyName    3143 non-null   object 
 3   StateName     3143 non-null   object 
 4   CountyFIPS    3143 non-null   object 
 5   StateAbbr     3143 non-null   object 
 6   STATE_COUNTY  3143 non-null   object 
 7   CountyCBSA    1835 non-null   float64
dtypes: float64(1), object(7)
memory usage: 196.6+ KB


In [22]:
value_counts_df(df)

Year
2017    3392
2019    3086
2016    3003
2015    2932
2021    2840
2018    2820
2013    2763
2014    2678
2012    2500
2011    2495
2010    2378
2009    2376
2020    2321
2008    2029
Name: count, dtype: int64
State
PA       5516
WI       5153
NY       4759
MN       3296
NJ       2872
VA       2422
MD       2155
ME       1311
MA       1092
WV       1084
CT        955
NH        943
VT        866
MI        681
IA        583
OH        536
RI        518
IL        490
NC        470
FL        446
DE        298
IN        265
CA        260
TX        160
OR        104
DC         90
ND         75
SC         52
NV         29
WA         26
AZ         25
UT         23
AL         16
TN         12
KS          9
KY          7
GA          5
ID          3
NM          2
NE          2
AK          2
Name: count, dtype: int64
FIPS 
34027    168
34005    156
34031    155
34013    154
34003    154
        ... 
48367      1
18047      1
18167      1
51678      1
01049      1
Name: count, Length: 782, dtype:

Year looks good, State looks good, FIPS looks good, Case_Status always looked good, Sex and Age_cat_yrs still has Suppressed and Unknown, but here it makes sense for me to turn that turn Suppressed into Unknown and keeping it instead of dropping it.

In [23]:
# Replacing all Suppressed with Unknown
df.replace(to_replace='Suppressed', value='Unknown', inplace=True)
value_counts_df(df)

Year
2017    3392
2019    3086
2016    3003
2015    2932
2021    2840
2018    2820
2013    2763
2014    2678
2012    2500
2011    2495
2010    2378
2009    2376
2020    2321
2008    2029
Name: count, dtype: int64
State
PA       5516
WI       5153
NY       4759
MN       3296
NJ       2872
VA       2422
MD       2155
ME       1311
MA       1092
WV       1084
CT        955
NH        943
VT        866
MI        681
IA        583
OH        536
RI        518
IL        490
NC        470
FL        446
DE        298
IN        265
CA        260
TX        160
OR        104
DC         90
ND         75
SC         52
NV         29
WA         26
AZ         25
UT         23
AL         16
TN         12
KS          9
KY          7
GA          5
ID          3
NM          2
NE          2
AK          2
Name: count, dtype: int64
FIPS 
34027    168
34005    156
34031    155
34013    154
34003    154
        ... 
48367      1
18047      1
18167      1
51678      1
01049      1
Name: count, Length: 782, dtype:

Success, we have a clean dataframe, next step is visualizing the data

As I decided to a with a web app without machine modeling which is possible according to the rubric I need to make sure that the data can be easily accessed in the web app. At the moment there is no real metric involved in the web app except the maximum frequency. This number will be determined in app.py. The next lines are for preparing the data and adding three total column for each line for the case status, age category and sex.

In [ ]:
# Looked like an elegant solution, but did not work :-(, I wanted to separated for each year. But now I am doing it in the
# web app with loc with works better.
for i, new_df_name in enumerate(new_df_names):
    print(i+2008, new_df_name)
    new_df_name = df[df['Year'] == i+2008]
    #new_df_name = pd.DataFrame(new_df_name)
    print(new_df_name)

Actually at the moment there is no need for the state column anymore and I am dropping it. For a better analysis one could compare the results between this way and dropping the the state column at the start and then
- dropping all Suppressed from the FIPS column
- dropping all Unknown from the FIPS column
- replacing all Suppressed with Unknown in the Dataframe

which is the pipeline I used here to clean the data.

In [24]:
# The way I massaged the dataframe up to now, there are no suppressed values in the FIPS colums, therefore I can drop the state
df.drop('State', axis=1, inplace=True)
df

,Year,FIPS,Case_status,Sex,Age_cat_yrs,Frequency
3,2008,04013,Confirmed,Male,0-19,2
4,2008,04013,Probable,Male,0-19,3
5,2008,04013,Probable,Male,Unknown,1
6,2008,04013,Probable,Unknown,Unknown,2
7,2008,06001,Confirmed,Unknown,Unknown,6
...,...,...,...,...,...,...
40453,2021,54107,Probable,Male,0-19,2
40454,2021,54107,Probable,Male,20+,4
40455,2021,54107,Probable,Female,20+,3
40456,2021,54109,Confirmed,Unknown,Unknown,3


To load the data into the web app, I am generating three .csv files containg
- df_case_status: columns Year, FIPS, Confirmed, Probable, Total
- df_sex: columns Year, FIPS, Female, Male, Unknown, Total
- df_age_cat_yrs: columns Year, FIPS, 0-19, 20+, Unknown, Total

To help with generating these .csv-files I wrote a function with takes the general dataframe, drops all the columns not need for that file, groups by year and FIPS and pivot the columns of interest into the value in this column. Also a total column is added which sum up over the columns for each year and county with data.

In [25]:
def generate_df_visualization(df, columns_to_drop, column_of_interest):
    """
    function which takes the df generated from the original data and transforms it into a dataframe more suitable for 
    visualization by taking only unmelting one column, dropping the rest and adding a total column, all depending on the
    county (FIPS)
    
    Parameters
    ----------
    df: dataframe
        dataframe to be transformed
    columns_to_drop: list of str
        list of columnsnames to be dropped because they are not needed for the visualization
    column_of interest: str
        column which needs to be transformed

    Returns
    -------
    dataframe
        transformed dataframe
    """
    
    # Dropping columns not needed
    df_transformed = df.drop(columns_to_drop, axis=1)
    # Columns to group by
    columns_to_group = ['Year', 'FIPS']
    columns_to_group.append(column_of_interest)
    df_transformed = df_transformed.groupby(by=columns_to_group).sum()
    df_transformed = df_transformed.reset_index()
    # Splits up the different values in the columns of intrest into different columns, giving it the frequency value
    df_transformed = df_transformed.pivot(index=['Year', 'FIPS'],
                                          columns=column_of_interest,
                                          values=['Frequency'])['Frequency']
    # Filling up the NaNs from the pivot with 0
    df_transformed = df_transformed.fillna(0).reset_index()
    # Adding total column
    # After all the transformation, after dropping Year and FIPS, only the columns to be summed over are left
    columns_to_sum = df_transformed.drop(['Year', 'FIPS'], axis = 1)
    df_transformed['Total'] = columns_to_sum.sum(axis = 1)
    return df_transformed


In [26]:
df_case_status = generate_df_visualization(df, ['Age_cat_yrs', 'Sex'], 'Case_status')

In [27]:
df_case_status

Case_status,Year,FIPS,Confirmed,Probable,Total
0,2008,04013,2.0,6.0,8.0
1,2008,06001,6.0,0.0,6.0
2,2008,06023,6.0,0.0,6.0
3,2008,06037,5.0,0.0,5.0
4,2008,06045,9.0,0.0,9.0
...,...,...,...,...,...
6440,2021,55133,151.0,60.0,211.0
6441,2021,55135,45.0,40.0,85.0
6442,2021,55137,23.0,13.0,36.0
6443,2021,55139,6.0,17.0,23.0


In [28]:
df_sex = generate_df_visualization(df, ['Case_status', 'Age_cat_yrs'], 'Sex')

In [29]:
df_sex

Sex,Year,FIPS,Female,Male,Unknown,Total
0,2008,04013,0.0,6.0,2.0,8.0
1,2008,06001,0.0,0.0,6.0,6.0
2,2008,06023,0.0,0.0,6.0,6.0
3,2008,06037,0.0,5.0,0.0,5.0
4,2008,06045,0.0,0.0,9.0,9.0
...,...,...,...,...,...,...
6440,2021,55133,75.0,136.0,0.0,211.0
6441,2021,55135,34.0,51.0,0.0,85.0
6442,2021,55137,18.0,18.0,0.0,36.0
6443,2021,55139,8.0,15.0,0.0,23.0


In [31]:
df_age_cat_yrs = generate_df_visualization(df, ['Case_status', 'Sex'], 'Age_cat_yrs')

In [32]:
df_age_cat_yrs

Age_cat_yrs,Year,FIPS,0-19,20+,Unknown,Total
0,2008,04013,5.0,0.0,3.0,8.0
1,2008,06001,0.0,0.0,6.0,6.0
2,2008,06023,0.0,0.0,6.0,6.0
3,2008,06037,0.0,0.0,5.0,5.0
4,2008,06045,0.0,0.0,9.0,9.0
...,...,...,...,...,...,...
6440,2021,55133,45.0,166.0,0.0,211.0
6441,2021,55135,16.0,69.0,0.0,85.0
6442,2021,55137,0.0,34.0,2.0,36.0
6443,2021,55139,0.0,21.0,2.0,23.0


To use these dataframes latter in the web app, they need to be saved in an /assets/data folder. The function below helps in making this saving a little bit easier.


In [33]:
# Defining function for saving the dataframe in the /assets/data folder
def save_dataframe_to_csv(dataframe, filename):
    # Constructing the filepath
    filepath = './assets/data/'+filename+'.csv'
    dataframe.to_csv(filepath, index=False)


In [34]:
save_dataframe_to_csv(df_age_cat_yrs, 'df_age_cat_yrs')

In [35]:
save_dataframe_to_csv(df_case_status, 'df_case_status')

In [36]:
save_dataframe_to_csv(df_sex, 'df_sex')

According to the rubric 

> If the student submits a web app rather than a blog post, then the Project Definition, Analysis, and Conclusion should be included in the README file, or in their Jupyter Notebook. Students should not use another student's code to complete the project, but they may use other references on the web including StackOverflow and Kaggle to complete the project. 

I included this information already in README file when I handed it in, but obviously it was not found and I will include the information also in here.



## Analysis

In this project for it is actually more important to learn how to deal with real world data than to actually draw conclusions.

This project clearly shows for me that it is always important to know where the data comes from and how it was processed before it is visualized in any way.

During the dataprocessing I realized that not every data point was split down to the county level, but was giving them on the state level. For my analysis I decided to drop these lines, but for a real world analysis a more thorough investigation would be needed, which would answer questions like: 
- If only the state is given, is it just a summary of the data for the state, then dropping would be fine, because otherwise data would be counted twice.
- Or are we loosing data by dropping these lines?
- As real people are involved, maybe they understood the questions differently and the answer depends on the state/clerk entering the data?

For the Suppressed and the Unknown values in the Age, Cases Status and Sex category a closer look could also be worthwile. Here I decided not to drop these lines but also add a Total column because otherwise too much information would be lost.

As real world data never is perfect it is always important to keep in the back of the head that in cleaning the data some information will always be lost.

It is also important to note that using modules and libraries it is relatively easy to draw this visualization, things still can go wrong and one would also need to think about plausibilty checks. I realized that Conneticut including Lyme is not represented on the visualization, even though it is in the data set. And given the high numbers there, it would completely change the visual impression.

What can be said about the data, with the caveats given above:
- Over time more and more counties have cases of Lyme disease, but mostly on the East Coast, around the Great Lakes and in the West.
- In the East a larger area is involved, but the number in the counties are lower. From this analysis one can not deduct if the total of cases have risen or fallen between 2008 and 2021.
- Especially in the East one need to be careful if talking about Confirmed, Probable or the sum of both.
- The gender does not make much of a difference.
- Lyme disease is more of a disease of grown-up (20+) than of children (0-19).

## Conclusion

The app is a good start to visualize the development of the number of Lyme disease cases in the US and especially to show the spread over the country over the years. And looking at a picture is a lot easier than looking at the bare numbers in a table.

The code developed now can be used as a framework for a lot of simular visualizations breaking information down into counties in the use.

On the other end I learned again that a lot of things can go wrong in a visualization like what happened to Conneticut and even though a visualization can be very suggestive, one also needs to question them.

# Improvements
Also there is still a lot to do to turn it into a really good visualization.
- Finding a better color scale
- Giving the option of changing the maximum number, at the moment it is fixed and is the maximum number of cases for the whole table.
- Centering everything.
- Different font.
- Clicking on a county and seeing the development of cases over the years in a graph.
- ...

When first thinking about this project I also had the idea to overlay the number of Lyme cases with how heavely wooded the counties are or how populated these areas are. But I could not find good data on that. But this would make the web app also more instructive or would be a good idea in the future for a machine learing model to see if the amount of forests correlate with the number of Lyme cases.

#### Reference for the code development
[Develop Data Visualization Interfaces in Python with Dash](https://realpython.com/python-dash/)

[Writing a simple plotly dash app](https://medium.com/@jandegener/writing-a-simple-plotly-dash-app-f5d83b738fd7)

[Dash Python User Guide](https://dash.plotly.com/), especially the Dash Fundamentals
